In [1]:
def generate_retrieval_data_without_hn(interactions, tokenizer, type):
    """
    Generate table retrieval data, without hard negatives.
    Return: 
    [
        {
            "question": "....",
            "answers": ["...", "...", "..."],
            "positive_ctxs": [{
                "title": "...",
                "text": "...."
            }]
        },
        ...
    ]
    """
    print("==== Now Generating table retrieval dataset without hard negatives. ====")

    tb_retrieval_data = []
    count = 0
    for it_object in tqdm(interactions):
        data = {}
        table = it_object["table"]
        title = table["documentTitle"]
        question = it_object["questions"][0]["originalText"]
        answers = it_object["questions"][0]["answer"]["answerTexts"]
        
        data["question"] = question
        data["answers"] = answers
        
        positive_context = {}
        linearized_column, psg_list, _, _, _  = gen_table_passages(table, tokenizer)
        gold_psg = ""
        max_answer_num = 0
        for psg in psg_list:
            psg = tokenizer.decode(tokenizer.convert_tokens_to_ids(psg))
            num_answers_found = answer_count(answers, psg, tokenizer)
            if num_answers_found > max_answer_num:
                max_answer_num = num_answers_found
                gold_psg = psg
        
        gold_text = linearized_column + " [SEP] " + gold_psg
        positive_context["title"] = title
        positive_context["text"] = gold_text
        data["positive_ctxs"] = [positive_context]
        tb_retrieval_data.append(data)

    if type == "train":
        with open("table_train.json", "w") as f:
            json.dump(tb_retrieval_data, f)
    elif type == "dev":
        with open("table_dev.json", "w") as f:
            json.dump(tb_retrieval_data, f)
    print("Saving of retrieval data done.")
    print(f"Number of q/a pairs: Train: {len(tb_retrieval_data)}")


In [3]:
import json
import pickle
import csv 
from transformers.tokenization_bert import BertTokenizer
from tqdm import tqdm


In [4]:
tok = BertTokenizer.from_pretrained("bert-base-uncased", lower=True)

In [6]:
b = tok.tokenize(text).words(uncased=True)
b

AttributeError: 'list' object has no attribute 'words'